In [1]:
import requests
import requests_cache
import pandas as pd
import itertools
from bs4 import BeautifulSoup
import pandas as pd
import math
import time

from bokeh.models import ColumnDataSource, LabelSet,HoverTool
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements

In [2]:
requests_cache.install_cache('craigslist_cache')

In [3]:
from data_function import *

# Data Preparation 

### Web Scraping 

In [59]:
df = get_all_search('cto', 'civic', 1999, 2000,1000,20000)
df.head()

,post_title,mileage,price,condition,title_status,transmission,drive,location,link
0,UNIQUE JDM 2000 CIVIC COUPE WITH JAPANESE FRON...,142000,3500,,clean,manual,,,https://sfbay.craigslist.org/sfc/cto/d/unique-...
1,1999 Honda Civic - Clean Title,,1600,,clean,automatic,,(portola district),https://sfbay.craigslist.org/sfc/cto/d/1999-ho...
2,2000 honda civic,,2000,good,salvage,automatic,fwd,(hayward / castro valley),https://sfbay.craigslist.org/eby/cto/d/2000-ho...
3,Honda civic 2000,166,1300,good,salvage,manual,,(sunnyvale),https://sfbay.craigslist.org/sby/cto/d/honda-c...
4,"2000 Honda Civic LX Sedan 4D (158,000 Miles) D...",158500,3450,excellent,clean,automatic,,(san jose west),https://sfbay.craigslist.org/sby/cto/d/2000-ho...


### Data Cleaning

In [60]:
df = df.replace(np.nan, 'Unknown', regex=True)
df.location = clean_location(df.location)
df.location = list(map(lambda x: x if len(x) != 0 else 'Unknown', df.location)) # Place Unknown if the location is empty
df.head()


C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  location_list[i] = re.sub(r'[()]', "", location_list[i])
C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  location_list[i] = location_list[i].strip(' ')
C:\Users\Jim\Desktop\Github\usedcaranalysis\data_function.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  location_list[i] = re.split( r'[\,]', location_lis

,post_title,mileage,price,condition,title_status,transmission,drive,location,link
0,UNIQUE JDM 2000 CIVIC COUPE WITH JAPANESE FRON...,142000,3500,,clean,manual,,Unknown,https://sfbay.craigslist.org/sfc/cto/d/unique-...
1,1999 Honda Civic - Clean Title,,1600,,clean,automatic,,Portola district,https://sfbay.craigslist.org/sfc/cto/d/1999-ho...
2,2000 honda civic,,2000,good,salvage,automatic,fwd,Hayward / castro valley,https://sfbay.craigslist.org/eby/cto/d/2000-ho...
3,Honda civic 2000,166,1300,good,salvage,manual,,Sunnyvale,https://sfbay.craigslist.org/sby/cto/d/honda-c...
4,"2000 Honda Civic LX Sedan 4D (158,000 Miles) D...",158500,3450,excellent,clean,automatic,,San jose west,https://sfbay.craigslist.org/sby/cto/d/2000-ho...


# Data Visualization

In [75]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import TextInput
from datetime import timedelta
from datetime import datetime as dt
from bokeh.models.widgets import MultiSelect, Slider
from bokeh.layouts import widgetbox, column
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, HoverTool, CustomJS


location=df["location"]
price = df["price"]
low = min(price)
high = max(price)
palette = ["#053061", "#2166ac", "#4393c3", "#92c5de", "#d1e5f0",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]
price_inds = [int(10*(x-low)/(high-low)) for x in price] 
df['price_colors'] = [palette[i] for i in price_inds]
annotations = [dict(
        showarrow = False,
        x = "2017-01-01",
        y = "A",
        z = 0,
        text = "Point 1",
        xanchor = "left",
        xshift = 10,
        opacity = 0.7
      )

TITLE = "Price v.s. Location"
TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

p = figure(y_range=list(set(location)), tools=TOOLS, toolbar_location="above", logo="grey", plot_width=1200, title=TITLE)
hover = p.select(dict(type=HoverTool))
hover.tooltips = [
    ("price", "@price"),
    ("location:", "@location"),
    ("title", "@post_title"),
    ("mileage", "@mileage"),
    ("condition", "@condition"),
    ("title_status", "@title_status"),
    ("transmission", "@transmission")
]

source = ColumnDataSource(df)

r=p.circle('price', 'location', size=15, source=source, color='price_colors', line_width=3)


def locsel(attr, old, new):
    vloc = multi_select.value
    df = df[df['location'].isin(vloc)]
    Currnew= ColumnDataSource(df)
    source.data =Currnew.data

multi_select = MultiSelect(title="Select:",value=['01'],  options=list(set(location)), height=400, width=100)
multi_select.on_change("value", locsel)

layout = column(multi_select, p)


output_file("Data Visualization.html")

show(layout)

W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='23ff4ee4-72b3-489a-b3c9-8840e73770b0', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='ad3be13d-0934-4395-880c-001f00586bfa', ...)
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: bokeh_size, density, pct_d [renderer: GlyphRenderer(id='c2f1ab0a-0bc6-4199-a47f-ec442d9097b2', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: bokeh_size, density, pct_r [renderer: GlyphRenderer(id='8cf452a4-a56a-49b0-bc2c-bf1b3bcc172a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: bokeh_size, density, pct_d [renderer: GlyphRenderer(id='391231a3-fe6e-49e3-a71e-1eba7f6e00e0', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: bokeh_size, density, pct_r [renderer: GlyphRenderer(id='d6216829-f512-4ef3-bf5d-93391bb29072', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: bokeh_size [renderer: GlyphRenderer(id=